# 7章 畳み込みニューラルネットワーク(convolutional neural network: CNN)
* CNNは画像認識や音声認識などいたるところで使われている
* 画像認識のコンペでは，ほとんどすべてがCNNベースである

## 7.1 全体の構造
* CNNもこれまでのニューラルネットワークと同じくレイヤを組み合わせて作る
* CNNでは「Convolutionレイヤ（畳み込み層）」と「Poolingレイヤ（プーリング層）」が加わる
* これまでのネットワークでは隣接する層の全ニューロン間が結合されていた（全結合：fully-connected）
  * 全結合層はAffineレイヤとして実装した
  * Affineレイヤの後にReLUを接続し，それを何回か繰り返したのち，最終出力では Affineレイヤ$\rightarrow$Softmaxレイヤの順で出力する
* CNNの場合の構成
  * Convolutionレイヤ$\rightarrow$ReLUレイヤ$\rightarrow$Poolingレイヤの順で接続し，それを何回か繰り返し，最終出力では全結合ネットワークと同様にAffineレイヤ$\rightarrow$Softmaxレイヤで出力する
  * Poolingレイヤは省略されることもある

## 7.2 畳み込み層
### 7.2.1 全結合層の問題点
* 全結合層のニューラルネットワークではAffineレイヤを使用した
* Affineレイヤの問題点はデータの形状が無視されること
  * 例えば，画像は本来「縦，横，チャネル」の方向の広がりを持った3次元形状のデータである
  * Affineレイヤに入力するためにはデータを1次元データに変換する必要がある
  * 画像の場合，隣接するピクセル間に類似性などがあったりするなどなんらかのパターンが含まれている
  * Affineレイヤはパターンを無視して入力データを扱い，パターンの情報を生かすことがない
* 畳み込み層は形状を維持する
  * 画像の場合では入力データを3次元データとして受け取り，3次元データとして次の層に出力する
    * CNNでは畳み込み層の入出力データのことを特徴マップ（feature map），入力特徴マップ，出力特徴マップのように呼ぶ場合がある
  * そのため，形状やパターンのデータをうまく扱える可能性が残る

### 7.2.2 畳み込み演算
* 畳み込み層では畳み込み演算を行う
  * 畳み込み演算は画像処理でいうところのフィルター演算に相当する
* 畳み込み演算は，「入力データ」に対して「フィルター」を適用する
  * 入力データおよびフィルタはともに縦横の形状をもつ
  * フィルタは「カーネル」と呼ばれることもある
* 畳み込み演算は入力データに対してフィルターのウィンドウを一定間隔でスライドさせながら適用させる
  * それぞれの場所でフィルタの要素と入力の対応する要素を乗算し，その和を計算する（積和演算）
  * その和の値を出力の対応する場所へ格納する
  * このプロセスをすべての場所で行うことで畳み込み演算の出力を得る
* Affineレイヤでは重みパラメータの他にバイアスが存在した．
* CNNの場合，フィルタのパラメータが重みに対応する．またバイアスも存在する
  * バイアスは(1,1)のデータで，フィルタ適用後の結果の全要素にそれぞれ加算される

### 7.2.3 パディング(padding)
* 出力サイズを調整するために，入力データの周囲に固定のデータを埋めること
  * 例えば「幅1のパディングを適用する」とは，周囲を幅1ピクセルの0で埋める
* パディングすると出力は大きくなる

### 7.2.4 ストライド(stride)
* フィルターを適用する位置の間隔
* 例えば，ストライドを2にすると，積和演算のためのフィルタは2要素ずつ移動する
* ストライドを大きくすると出力は小さくなる
* サイズの関係: $OH, OW$の計算結果は整数にならなければならないことに注意（割り切れること）
  * 入力サイズ $(H,W)$
  * フィルターサイズ $(FH,FW)$
  * 出力サイズ $(OH, OW)$
  * パディング $P$
  * ストライド $S$
$$OH=\frac{H+2P-FH}{S}+1 \\
OW=\frac{W+2P-FW}{S}+1$$

### 7.2.5 3次元データの畳み込み演算
* 画像の場合は縦横だけでなくチャネル方向も合わせた3次元で扱う必要がある
* チャネル方向も合わせた3次元データに対する畳み込み演算を考える
* チャネルごとに特徴マップがある場合，チャネルごとに入力データとフィルタの畳み込み演算を行い，チャネルごとの結果を加算して一つの出力を計算する
* 注意点：入力データとフィルタのチャネル数は同じにする
  * フィルタのサイズは自由に設定できる

### 7.2.6 ブロックで考える
* 前節では複数チャネル（チャネル数 $C$）からなる画像データ(サイズ $H \times W$)に，それと同じチャネル数($C$)のフィルタ(サイズ $FH \times FW$)を適用して一つの（チャネルが1の）出力特徴マップ(サイズ $OH \times OW$)を得た
* 複数チャネル($FN$個)からなる出力特徴マップを得るにはどうしたら良いか $\rightarrow$ 複数($FN$個)のフィルタを用いる
* 畳み込み演算のフィルタはフィルタの個数$(FN)$も考慮すると4次元のデータとなる
  * (出力チャネル数，入力チャネル数，出力縦，出力横)
    * チャネル数3, サイズ5$\times$5 のフィルタが20個ある場合は $(20, 3, 5, 5)$
* 畳み込み演算のバイアスは出力チャネル数を考慮すると (FN, 1, 1) となる
  * チャネルごとに同じバイアス値がチャネル内の全要素に加算される
* まとめ
  * 入力特徴マップ $(C, H, W)$
  * フィルタ $(FN, C, FH, FW)$
  * バイアス $(FN, 1, 1)$
  * 出力特徴マップ $(FN, OH, OW)$

### 7.2.7 バッチ処理
* バッチを考慮しない場合の入力特徴マップは $(C, H, W)$，出力は $(FN, OH, OW)$
* バッチ処理を考慮する場合，複数($N$個)のデータをまとめて処理することになり，それが畳み込み層へ入力される
* したがってバッチ処理を考慮すると，レイヤの特徴マップは4次元になる：入力 $(N, C, H, W)$ および出力 $(N, FN, OH, OW)$

## 7.3 プーリング層
* プーリングは縦・横方向の空間を小さくする演算
* 複数の要素を一つに集約するような処理を行うことで，空間を小さくする
* 最大値をとるMaxプーリングや平均をとるAverageプーリングなどがある
  * ここではMaxプーリングのみを使用する

### 7.3.1 プーリング層の特徴
* 学習するパラメータがない
  * 入力データだけを使って行う処理なので，学習が必要ない
* チャネル数は変化しない
  * チャネルごとに処理を行い，まとめないためチャネル数は変わらない
* 微小な位置変化に対してロバストである
  * 多少ずれても同じような結果となる（必ずしも同じにはならないことに注意）

In [1]:
import numpy as np

## 7.4 Convolution/Poolingレイヤの実装
* レイヤなのでこれまでと同様，forward と backward というメソッドを持つ

### 7.4.1 4次元配列
* CNNの各層を流れるデータは（バッチおよびチャネルを考慮して）4次元データである

In [5]:
# 適当な4次元配列の生成
# バッチサイズ10，チャネル1，縦横それぞれ28のデータとして生成してみる
x = np.random.rand(10, 1, 28, 28)
# 形状を確認
print(x.shape)

# 一つ目のデータにアクセス
#print(x[0])
# 一つ目の1チャネル目のデータにアクセス
print(x[0][0])

(10, 1, 28, 28)
[[  2.19232042e-01   5.90861140e-01   2.63953204e-01   4.18263379e-01
    6.08842501e-01   6.79750881e-01   9.91308545e-01   4.34468324e-01
    7.68900295e-01   3.15335981e-01   6.94250983e-01   6.95741022e-01
    3.78626632e-02   6.45433312e-01   1.67335035e-01   5.42430911e-01
    1.92928840e-01   8.97749645e-01   1.05179618e-01   7.06824953e-02
    7.54502595e-01   2.32788920e-01   7.20376113e-01   1.29690383e-01
    3.92271349e-01   9.63076902e-01   2.00461682e-01   4.10595885e-01]
 [  6.16314215e-02   7.66184999e-01   4.17424746e-01   5.31912690e-01
    4.29193874e-01   3.29056436e-01   4.52644242e-01   1.34644337e-01
    3.62031826e-01   6.61525888e-01   9.72074578e-01   9.77747470e-01
    9.37942721e-01   5.13356046e-01   8.15957317e-01   2.01714083e-01
    1.96431033e-01   5.02608040e-01   8.94071209e-02   6.07131982e-01
    9.72602574e-01   4.36672643e-01   8.46574074e-01   5.24599400e-01
    8.12844564e-01   6.88401405e-01   7.92341390e-01   7.21485930e-01]
 [

### 7.4.2 im2col による展開
* 畳み込み演算を愚直に実装するとforを多段にネストしたコードになるが，Numpyだとパフォーマンス上問題になる
* for の代わりに im2col という関数を使用する

In [9]:
# im2col (common/util.py より)
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    
    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング
    
    Returns
    -------
    col : 2次元配列
    """
    # 入力データのデータ数，チャネル数，縦，横を取得
    N, C, H, W = input_data.shape
    # 出力データの縦，横のサイズを算出
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    
    # pad関数は配列に詰め物をする．ここでは3番目の軸と4番目の軸の両端にそれぞれpad個ずつ定数を追加する．
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    print(img.shape)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
    
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


In [18]:
x = np.random.rand(10, 1, 28, 28)
print(x.shape)
pad = 0
img = np.pad(x, [(0,1), (1,0), (pad, pad), (pad, pad)], 'constant')
print(img.shape)

(10, 1, 28, 28)
(11, 2, 28, 28)


### 7.4.3 Convolutionレイヤの実装
### 7.4.4 Poolingレイヤの実装
## 7.5 CNNの実装
## 7.6 CNNの可視化
### 7.6.1 1層目の重みの可視化
### 7.6.2 階層構造による情報抽出

## 7.7 代表的なCNN
* CNNは様々な構成のネットワークが提案されてきた
* LeNetおよびAlexNetは特に重要なネットワークである

### 7.7.1 LeNet
* CNNの元祖
* 手書き数字認識を行うネットワークとして1998年に提案された
* 畳み込み層とサブサンプリング層（要素を間引く層）を連続して行い，最後に全結合層を経て結果が出力される
* 現在のCNNとの違い
  * シグモイド関数が使われている（現在は主にReLUが使われる）
  * サブサンプリングによって中間データのサイズを縮小している（現在はMaxプーリングが主流） 

### 7.7.2 AlexNet
* ディープラーニングが注目されるに至ったCNN．2012年に提案された．
* 畳み込み層とプーリング層を重ね，最後に全結合層を経由して結果を出力する
* 基本構成はLeNetと変わらないが，以下の点で異なる
  * ReLUを使用する
  * LRN(Local Response Normalization)という局所的正規化の層を使用する
  * Dropoutを使用する
* LeNetの時代と比べるとコンピューティング環境が変化
  * 大量データの入手が容易になった
  * GPUが普及し，大量の演算を高速に実行可能となった

## 7.8 まとめ
* CNNについて学んだ
  * CNNを構成する基本モジュールは「畳み込み層」と「プーリング層」である
  * CNNはこれまでの全結合層のネットワークに加え，畳み込み層とプーリング層が加わったもの
  * CNNは画像を扱う分野では例外なく使われる
* 畳み込み層とプーリング層はim2colを用いるとシンプルで効率よく実装できる
* CNNの可視化により高度な情報が層を深くするにつれて抽出される様子がわかる
* ディープラーニングの発展にビックデータとGPUが大きく貢献している